In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

Data Loading

In [44]:

def load_data(file):
    mdro = pd.read_excel(file, sheet_name='MDROs', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    report = pd.read_excel(file, sheet_name='微生物報告', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    info = pd.read_excel(file, sheet_name='就醫資訊', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    # anti = pd.read_excel(file, sheet_name='抗生素使用', 
    #                     dtype={'病歷號碼':'str', '就醫序號':'int'})
    examValue = pd.read_excel(file, sheet_name='檢驗數值', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    
    # data processing 
    mdro = mdro[['病歷號碼', 'time', '就醫序號','申請編號', '菌株', '預測值', '部位', 'sendTime']]
    mdro.rename({'time':'openTime'}, axis=1, inplace=True)
    
    report = report[['病歷號碼', 'bed', '就醫序號','申請編號', 'strain','部位', '抗藥性菌株', 'examTime','reportTime']]
    
    cdss = pd.merge(report, mdro, how='left', on=['病歷號碼', '就醫序號','申請編號', '部位'])
    cdss = cdss.drop_duplicates()
    cdss['miniTime_Open'] = cdss.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值',  'sendTime'])['openTime'].transform('min')
    
    info = info[['病歷號碼', '就醫序號', '性別', '生日', '年齡', '來院日期', '離院日期', '天數', '門急住', '科別代碼', '科別']]
    info.rename({'門急住':'bed'}, axis=1, inplace=True)
    
    cdss_info = pd.merge(cdss, info, how='left', on=['病歷號碼', '就醫序號', 'bed'])
    
    # Filtering mini open CDSS time
    CDSS = cdss_info[cdss_info.openTime==cdss_info.miniTime_Open].drop_duplicates()
    
    # Add column : max time leaving hospital
    CDSS.離院日期 = pd.to_datetime(CDSS.離院日期).dt.date
    CDSS['MaxTime_leave'] = CDSS.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值', 'sendTime', '來院日期'])['離院日期'].transform('max')
    # CDSS = pd.merge(cdss_info, anti, how='left', on=['病歷號碼','就醫序號'])
    # CDSS = CDSS.drop_duplicates()
    
    # Predict Level
    condition_1 = [(CDSS['預測值'] <= 20), (CDSS['預測值'] > 20)&(CDSS['預測值'] <= 40), 
             (CDSS['預測值'] > 40)&(CDSS['預測值'] <= 60), (CDSS['預測值'] > 60)&(CDSS['預測值'] <= 80),
             (CDSS['預測值'] > 80)&(CDSS['預測值'] <= 100)]

    value_1 = ['0~20%', '21~40%', '41~60%', '61~80%', '81~100%']
    CDSS['PredLevel'] = np.select(condition_1, value_1) 
    
    # Stain Category
    condition_2 =[
                  (CDSS['菌株'].isin(['Enterococcus faecium(VRE)', 'Enterococcus faecium', 'Enterococcus faecalis'])),
                  (CDSS['菌株'].isin(['Morganella morganii'])),
                  (CDSS['菌株'].isin(['Klebsiella pneumoniae'])),
                  (CDSS['菌株'].isin(['Pseudomonas aeruginosa'])),
                  (CDSS['菌株'].isin(['Escherichia coli'])),
                  (CDSS['菌株'].isin(['Acinetobacter baumannii', 'Acinetobacter baumannii/calcoaceticus complex'])),
                  (CDSS['菌株'].isin(['Staphylococcus aureus', 'Staphylococcus aureus(MRSA)'])),
                  (CDSS['菌株'].isin(['Enterobacter cloacae']))
                ]
    
    value_2 = ['Enterococcus faecium', 'Morganella morganii', 'Klebsiella pneumoniae', 'Pseudomonas aeruginosa',
               'Escherichia coli', 'Acinetobacter baumannii', 'Staphylococcus aureus', 'Enterobacter cloacae']
    
    CDSS['Group_Strain'] = np.select(condition_2, value_2) 
    
    # exam Year & Month
    CDSS["examDate"] = pd.to_datetime( 
                                  pd.to_datetime(CDSS.examTime).dt.year.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.month.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.day.map(str)
                                )
    
    CDSS['Resist_bool'] = np.where(CDSS['抗藥性菌株']==1, True, False)
    CDSS.rename({'年齡':'Age', '預測值':'PredValue'}, axis=1, inplace=True)
    
    return CDSS
    

In [45]:
# 智抗菌平台報表_20220331.xlsx

file = r'C:\Users\User\VSCode\智抗菌平台報表_20220331.xlsx'
CDSS = load_data(file)
# CDSS.head(10)

In [46]:
old_file = r'C:\Users\User\VSCode\智抗菌平台報表_20220309.xlsx'
CDSS_old = load_data(old_file)
CDSS = pd.concat([CDSS_old, CDSS])
CDSS = CDSS.drop_duplicates().reset_index(drop=True).sort_values(['病歷號碼', '申請編號', 'examTime'])

In [47]:
# CDSS.to_excel(r'C:\Users\User\VSCode\CDSS_report_mdro_info.xlsx')

In [40]:
SMS = pd.read_excel(r'C:\Users\User\VSCode\簡訊通知時間(0301-0331)給鈞威.xls', dtype={'病歷號_10碼':'str'},
                    sheet_name='0301-0309' )

In [58]:
SMS = SMS[['西元年月', '時間', '病人姓名', '病歷號_10碼', '西元採檢日']]
SMS.rename({'病歷號_10碼':'病歷號', '西元採檢日':'examDate'}, axis=1, inplace=True)


In [59]:
SMS

,西元年月,時間,病人姓名,病歷號,examDate
0,2022-03-01,13:28:00,陳龍鑫,0017662463,2022-02-25
1,2022-03-01,13:28:00,黃裕益,0038131412,2022-02-25
2,2022-03-01,13:28:00,陳俊吉,0030442219,2022-02-25
3,2022-03-01,13:28:00,周嘉豪,0031487807,2022-02-26
4,2022-03-01,15:51:00,周嘉豪,0031487807,2022-02-28
...,...,...,...,...,...
119,2022-03-31,14:35:00,向富義,0037818664,2022-03-30
120,2022-03-31,14:35:00,陳榮梅,0019781710,2022-03-30
121,2022-03-31,14:35:00,林信有,0000944590,2022-03-29
122,2022-03-31,14:35:00,陳黃淑仔,0038128193,2022-03-30


In [36]:
CDSS.離院日期 = pd.to_datetime(CDSS.離院日期).dt.date

In [42]:
c = CDSS[CDSS.MaxTime_leave == CDSS.離院日期]

In [43]:
anti = pd.read_excel(file, sheet_name='抗生素使用', 
                     dtype={'病歷號碼':'str', '就醫序號':'int'})

anti

,病歷號碼,就醫序號,類別,醫令代碼,每次量,途徑,頻率,使用天數,開始時間,截止時間,囑咐
0,0031795531,161572515,住院用藥,ITEICOP,2.00,IVD,STAT,1,2020-07-02 10:06,2020-07-02 10:06,sent to OR 釋液:NS;總量:30mL;時間:30分;流速:60mL/hr;
1,0031795531,161572515,住院用藥,ICEFEPI,4.00,IVD,STAT,1,2020-07-02 10:06,2020-07-02 10:06,sent to OR 釋液:NS;總量:60mL;時間:30分;流速:120mL/hr;
2,0031795531,161572515,住院用藥,ICEFEPI,4.00,IVD,STAT,1,2020-08-19 13:59,2020-08-19 13:59,prepare heart transplantation釋液:NS;總量:60mL;時間:...
3,0031795531,161572515,住院用藥,ISEPTRI,2.00,IVD,STAT,1,2020-08-19 13:59,2020-08-19 13:59,prepare heart transplantation釋液:NS;總量:125mL;時間...
4,0031795531,161572515,住院用藥,IDAPTOM,1.00,IVD,STAT,1,2020-08-31 09:32,2020-08-31 09:32,s/p VAD釋液:NS;總量:50mL;時間:30分;流速:100mL/hr;
...,...,...,...,...,...,...,...,...,...,...,...
12665,0026313043,233386061,住院用藥,ISULAMP,2.00,IVD,Q6H,5,2022-03-31 22:06,2022-04-05 22:05,釋液:NS;總量:60mL;時間:15分;流速:240mL/hr;for cellulitis
12666,0015713331,233386229,住院用藥,ISOONME,1.00,IVD,Q8H,5,2022-03-31 21:10,2022-04-05 21:09,釋液:NS;總量:100mL;時間:30分;流速:200mL/hr;for supraglo...
12667,0038233494,233386293,住院用藥,IAMPICI5,0.32,IVD,Q8H-7,5,2022-03-31 22:14,2022-04-05 22:13,釋液:NS;時間:10分;流速:10mL/hr;
12668,0038233494,233386293,住院用藥,IGENTAM3,0.15,IVD,HS,5,2022-03-31 22:14,2022-04-05 22:13,釋液:NS;時間:60分;流速:10mL/hr;
